# ResNet for CIFAR-10

We take the inspiration for this code from the github link `pytorch-cifar`. It provides the basic block implementation of a Residual Network architecture. In our project, we declare a ResNet12 architecture, with four layers and `[2, 1, 1, 1]` blocks in each layer.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, dropout_rate=0.25):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(dropout_rate)  # Add dropout
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)  # Apply dropout after first activation
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, dropout_rate=0.5):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, dropout_rate=dropout_rate)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, dropout_rate=dropout_rate)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, dropout_rate=dropout_rate)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, dropout_rate=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)  # Add dropout before the final layer
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride, dropout_rate):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, dropout_rate))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)  # Apply dropout before the final fully connected layer
        out = self.linear(out)
        return out

def ResNet12():
    return ResNet(BasicBlock, [2, 1, 1, 1])


Next, we define a few helper functions to import data and format training progress. Some are also taken from the `python-cifar` utility functions.

In [2]:
import os
import sys
import time
import math
import zipfile
import pickle
import torch.nn as nn
import torch.nn.init as init


term_width = 50
last_time = time.time()
begin_time = last_time

def progress_bar(current, total, msg=None):
    bar_length = 60
    progress = current / total

    block = int(round(bar_length * progress))
    # Use '>' as an arrow to indicate current progress position
    # Note: The arrow is positioned at the end of the filled portion, except when progress is 0
    arrow = ">" if block < bar_length else ""
    text = "\rProgress: [{0}{1}{2}] {3:.2f}% ({4}/{5}) {6}".format(
        "=" * (block - 1 if block > 0 else 0), arrow,
        "-" * (bar_length - block), progress * 100, current, total,
        msg if msg else "")
    sys.stdout.write(text)
    sys.stdout.flush()
    if current == total:
        sys.stdout.write('\n')

def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    secondsf = int(seconds)
    seconds = seconds - secondsf
    millis = int(seconds*1000)

    f = ''
    i = 1
    if days > 0:
        f += str(days) + 'D'
        i += 1
    if hours > 0 and i <= 2:
        f += str(hours) + 'h'
        i += 1
    if minutes > 0 and i <= 2:
        f += str(minutes) + 'm'
        i += 1
    if secondsf > 0 and i <= 2:
        f += str(secondsf) + 's'
        i += 1
    if millis > 0 and i <= 2:
        f += str(millis) + 'ms'
        i += 1
    if f == '':
        f = '0ms'
    return f


def unpickle(filename):
    zf = zipfile.ZipFile(filename, "r")
    data = pickle.loads(zf.open("cifar_test_nolabels.pkl").read())
    zf.close()
    return data

Finally, we instanciate our model and count the number of parameters. We also import the CIFAR-10 dataset.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn.init as init
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import argparse
import sys
import time
import math
from PIL import Image

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

class CIFARLikeDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.data = []
        self.labels = []
        self.transform = transform

        # Load all batch files
        for i in range(1, 6):
            file_path = os.path.join(folder_path, f'data_batch_{i}')
            with open(file_path, 'rb') as file:
                batch = pickle.load(file, encoding='bytes')
                self.data.append(batch[b'data'])
                self.labels.extend(batch[b'labels'])

        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # reshape and transpose to HWC format

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]
        
        # Convert numpy array to PIL Image
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        return img, label
    
custom_data_path = './data'  # Update this path
cifar_like_train = CIFARLikeDataset(folder_path=custom_data_path, transform=transform_train)


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
combined_set = torch.utils.data.ConcatDataset([trainset, testset, cifar_like_train])
print(len(trainset), len(combined_set))
trainloader = torch.utils.data.DataLoader(
    combined_set, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet12()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(net)
count_params = count_parameters(net)
print("Count params: ", count_params)
assert count_params < 5000000, "Parameters must be lower than 5 million"

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
50000 110000
==> Building model..
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.5, inplace=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tr

# Training
For training, we use `CrossEntropyLoss` to meausre how the model is doing. This is a suitable loss for an image classification task. Next, we use the `Adam` optimizer to train the parameters. The learning rate will be updated using cosine annealing schedule.

In [4]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# Loss
criterion = nn.CrossEntropyLoss()
# Training Optimizer
optimizer = optim.Adam(net.parameters())
# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

# History
train_loss_hist = []
test_loss_hist = []

# Training
def train(epoch, dataloader):
    global best_acc
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx+1, len(dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    # save the history
    train_loss_hist.append(train_loss)
    # Save checkpoint.
    acc = 100.*correct/total
    if acc >= best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc


def test(epoch, dataloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            progress_bar(batch_idx+1, len(dataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # save the history
    test_loss_hist.append(test_loss)

Do 25 epochs of training.

In [ ]:
# Do 25 epoch
for epoch in range(start_epoch, start_epoch+500):
    train(epoch, trainloader)
    test(epoch, testloader)
    scheduler.step()


Epoch: 0
Progress: [===========================================================] 100.00% (860/860) Loss: 1.315 | Acc: 51.830% (57013/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 1.122 | Acc: 60.120% (6012/10000)

Epoch: 1
Progress: [===========================================================] 100.00% (860/860) Loss: 0.919 | Acc: 66.989% (73688/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.818 | Acc: 70.420% (7042/10000)

Epoch: 2
Progress: [===========================================================] 100.00% (860/860) Loss: 0.783 | Acc: 72.011% (79212/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.655 | Acc: 76.240% (7624/10000)

Epoch: 3
Progress: [===========================================================] 100.00% (860/860) Loss: 0.698 | Acc: 75.170% (82687/110000)
Saving..
Progre

Progress: [===========================================================] 100.00% (860/860) Loss: 0.186 | Acc: 93.325% (102658/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.034 | Acc: 99.410% (9941/10000)

Epoch: 30
Progress: [===========================================================] 100.00% (860/860) Loss: 0.180 | Acc: 93.537% (102891/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.031 | Acc: 99.460% (9946/10000)

Epoch: 31
Progress: [===========================================================] 100.00% (860/860) Loss: 0.176 | Acc: 93.706% (103077/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.031 | Acc: 99.590% (9959/10000)

Epoch: 32
Progress: [===========================================================] 100.00% (860/860) Loss: 0.172 | Acc: 93.842% (103226/110000)
Saving..
Progress:

Progress: [===========================================================] 100.00% (860/860) Loss: 0.097 | Acc: 96.549% (106204/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.007 | Acc: 99.950% (9995/10000)

Epoch: 59
Progress: [===========================================================] 100.00% (860/860) Loss: 0.096 | Acc: 96.630% (106293/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.007 | Acc: 99.940% (9994/10000)

Epoch: 60
Progress: [===========================================================] 100.00% (860/860) Loss: 0.094 | Acc: 96.616% (106278/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.006 | Acc: 99.930% (9993/10000)

Epoch: 61
Progress: [===========================================================] 100.00% (860/860) Loss: 0.095 | Acc: 96.579% (106237/110000)
Progress: [================

Progress: [===========================================================] 100.00% (860/860) Loss: 0.060 | Acc: 97.873% (107660/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.002 | Acc: 100.000% (10000/10000)

Epoch: 88
Progress: [===========================================================] 100.00% (860/860) Loss: 0.058 | Acc: 97.956% (107752/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.002 | Acc: 100.000% (10000/10000)

Epoch: 89
Progress: [===========================================================] 100.00% (860/860) Loss: 0.058 | Acc: 97.941% (107735/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.001 | Acc: 100.000% (10000/10000)

Epoch: 90
Progress: [===========================================================] 100.00% (860/860) Loss: 0.058 | Acc: 97.974% (107771/110000)
Saving..
Progress: [==========

Progress: [===========================================================] 100.00% (860/860) Loss: 0.038 | Acc: 98.666% (108533/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 117
Progress: [===========================================================] 100.00% (860/860) Loss: 0.038 | Acc: 98.669% (108536/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 118
Progress: [===========================================================] 100.00% (860/860) Loss: 0.037 | Acc: 98.690% (108559/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 119
Progress: [===========================================================] 100.00% (860/860) Loss: 0.034 | Acc: 98.807% (108688/110000)
Saving..


...

Progress: [===========================================================] 100.00% (860/860) Loss: 0.017 | Acc: 99.411% (109352/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 204
Progress: [===========================================================] 100.00% (860/860) Loss: 0.017 | Acc: 99.445% (109389/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 205
Progress: [===========================================================] 100.00% (860/860) Loss: 0.016 | Acc: 99.485% (109433/110000)
Saving..
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 206
Progress: [===========================================================] 100.00% (860/860) Loss: 0.016 | Acc: 99.419% (109361/110000)
Progress: [================

Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 233
Progress: [===========================================================] 100.00% (860/860) Loss: 0.016 | Acc: 99.440% (109384/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 234
Progress: [===========================================================] 100.00% (860/860) Loss: 0.017 | Acc: 99.423% (109365/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 235
Progress: [===========================================================] 100.00% (860/860) Loss: 0.017 | Acc: 99.411% (109352/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 236
Progress: [==============

Progress: [===========================================================] 100.00% (860/860) Loss: 0.021 | Acc: 99.274% (109201/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 263
Progress: [===========================================================] 100.00% (860/860) Loss: 0.020 | Acc: 99.288% (109217/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 264
Progress: [===========================================================] 100.00% (860/860) Loss: 0.020 | Acc: 99.302% (109232/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 265
Progress: [===========================================================] 100.00% (860/860) Loss: 0.021 | Acc: 99.280% (109208/110000)
Progress: [=========================

Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 292
Progress: [===========================================================] 100.00% (860/860) Loss: 0.030 | Acc: 98.963% (108859/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 293
Progress: [===========================================================] 100.00% (860/860) Loss: 0.029 | Acc: 99.008% (108909/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 294
Progress: [===========================================================] 100.00% (860/860) Loss: 0.030 | Acc: 98.975% (108873/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.000 | Acc: 100.000% (10000/10000)

Epoch: 295
Progress: [==============

Progress: [===========================================================] 100.00% (860/860) Loss: 0.041 | Acc: 98.585% (108444/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.001 | Acc: 100.000% (10000/10000)

Epoch: 322
Progress: [===========================================================] 100.00% (860/860) Loss: 0.040 | Acc: 98.604% (108464/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.001 | Acc: 100.000% (10000/10000)

Epoch: 323
Progress: [===========================================================] 100.00% (860/860) Loss: 0.040 | Acc: 98.629% (108492/110000)
Progress: [===========================================================] 100.00% (100/100) Loss: 0.001 | Acc: 100.000% (10000/10000)

Epoch: 324
Progress: [===========================================================] 100.00% (860/860) Loss: 0.040 | Acc: 98.599% (108459/110000)
Progress: [=========================

We can observe the loss over time.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,6))
plt.plot(train_loss_hist, '-', linewidth=3, label='Train Error')
plt.plot(test_loss_hist, '-', linewidth=3, label='Test Error')
plt.legend()
plt.grid(True)
plt.show()

We also can see the prediction performance on the test data

In [ ]:
from sklearn.metrics import confusion_matrix

labels = []
with torch.no_grad():
    for batch_idx, (input_test,label_test) in enumerate(testloader):
        input_test = input_test.to(device)
        outputs = net(input_test.float())
        _, predicted = outputs.max(1)
        labels.append(predicted)

# flatten the labels
pred = np.array([])
for batch in labels:
    pred = np.append(pred, batch.tolist())

# predict performance
cf_matrix = confusion_matrix(testset.targets, pred)
cf_matrix

It looks like all labels have around 90% accuracy score. We see that label 2 ('bird') has the lowest accuracy of 88.7%. Let's look at some wrong prediction examples:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

print_testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True)

fig, ax = plt.subplots(1, 3, figsize=(15, 10))
count = 0

for i in range(0, len(pred)):
    if print_testset.targets[i] != pred[i]:
        # print the image
        im = print_testset[i][0]
        ax[count].imshow(im)
        ax[count].set_title("Act:%s Pred:%s" % (classes[print_testset.targets[i]], classes[int(pred[i])]))
        count += 1

    if count == 3:
        break

plt.show()

For our best result, we trained the model using the combined dataset for 100 epochs with the RTX8000 GPU is NYU's HPC cluster.

In [ ]:
# Do 100 epoch
for epoch in range(start_epoch, start_epoch+100):
    train(epoch, trainloader)
    test(epoch, testloader)
    scheduler.step()

# Prediction
For Kaggle's data prediction, we will import the trained model and start predicting.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.nn.init as init

import torchvision
import torchvision.transforms as transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Model
print('==> Building model..')
net = ResNet12()
net = net.to(device)
net = torch.nn.DataParallel(net)
if device == 'cuda':
    cudnn.benchmark = True

# load state
saved_model = torch.load('./checkpoint/ckpt.pth', map_location=torch.device(device))
net.load_state_dict(saved_model['net'])
net.eval()

Preprocess the raw data and predict in batches of a thousand.

In [ ]:
import numpy as np
from PIL import Image

cifar_test_nolabels = unpickle("../data/cifar_test_nolabels.pkl.zip")
competition_data = cifar_test_nolabels[b'data']
y_id = cifar_test_nolabels[b'ids']

competitionset = []
for d in competition_data:
    # transpose flatten CIFAR image to RGB
    d = d.reshape(3, 32, 32).transpose(1,2,0)
    im = Image.fromarray(d, mode='RGB')
    im = transform_test(im)
    competitionset.append(im)

competitionloader = torch.utils.data.DataLoader(
    competitionset, batch_size=1000, shuffle=False)

labels = []

def competition_test():
    net.eval()
    
    with torch.no_grad():
        for batch_idx, input_test in enumerate(competitionloader):
            input_test = input_test.to(device)
            outputs = net(input_test.float())
            _, predicted = outputs.max(1)
            labels.append(predicted)
            print("Batch %d done" % (batch_idx + 1))

            
competition_test()

Finally, we write the prediction labels into a .csv file.

In [ ]:
import pandas as pd

# flatten the labels
pred = np.array([])
for batch in labels:
    pred = np.append(pred, batch.tolist())


competition_data_ids = unpickle("../data/cifar_test_nolabels.pkl.zip")[b'ids']
    
# create dataframe to save prediction
df = pd.DataFrame()
df['ID'] = competition_data_ids
df['Label'] = pred
df['Label'] = df['Label'].astype(int)

df.to_csv('out.csv', index=False)